# **Hyperparameter Tuning with Optuna (Tabnet)**

## 1. **Defining the Search Space and Launching the Optimization Study**

In this section, we tune the TabNet model using **Optuna** to automatically search for the best hyperparameters that maximize validation F1-score.

We used:
- `optuna.create_study(direction="maximize")` to find hyperparameter combinations that boost F1
- A search space including TabNet dimensions (`n_d`, `n_a`), number of steps, `gamma`, `lambda_sparse`, learning rate, etc.
- `torch.optim.lr_scheduler.StepLR` as our learning rate scheduler
- A custom objective function that returns validation F1-score

We saved:
- The full Optuna study (`tabnet_study.pkl`)
- The best hyperparameter configuration (`tabnet_best_params.csv`)

This step follows best practices from **Chapter 7 – Advanced Deep Learning** in *François Chollet’s Deep Learning with Python*, especially around hyperparameter optimization and model generalization.


In [ ]:
# tuning/tabnet_tuning.ipynb
import sys
import os

# Set path to root of the project (adjust if needed)
project_root = os.path.abspath("..")  # one level up
if project_root not in sys.path:
    sys.path.append(project_root)

In [15]:
# 1. Imports and Setup
import optuna
import joblib
import os
import numpy as np
import pandas as pd
import torch



from models.tabnet_model import build_tabnet_model
from training.tabnet_trainer import train_tabnet_model
from sklearn.metrics import f1_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
from utils.style_utils import styled_print
from torch.optim import Adam, RMSprop
from torch.optim.lr_scheduler import StepLR  



# 2. Load the original non-SMOTE, scaled dataset
data_path = "../artifacts/tabnet/data_scaled_nosmote_for_tabnet.pkl"
X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test = joblib.load(data_path)

styled_print("🗂️ Loaded the original non-SMOTE, scaled dataset from: artifacts/tabnet/data_scaled_nosmote_for_tabnet.pkl")

#  3. Define Optuna Objective Function
def objective(trial):
    params = {
        "n_d": trial.suggest_categorical("n_d", [8, 16, 32, 64]),
        "n_a": trial.suggest_categorical("n_a", [8, 16, 32, 64]),
        "n_steps": trial.suggest_int("n_steps", 3, 7),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0, step=0.5),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-5, 1e-2, log=True),
        "optimizer_params": dict(lr=trial.suggest_float("lr", 1e-4, 1e-2, log=True)),
        "scheduler_params": {"step_size": 10, "gamma": 0.95},
        "scheduler_fn": torch.optim.lr_scheduler.StepLR,
        "mask_type": "entmax",  # better for sparse data
        "device_name": "cuda" if torch.cuda.is_available() else "cpu",
    }

    model = TabNetClassifier(**params)
    model.fit(
        X_train=X_train_scaled.values,
        y_train=y_train.values,
        eval_set=[(X_val_scaled.values, y_val.values)],
        eval_metric=["auc", "f1_score"],
        patience=10,
        max_epochs=50,
        batch_size=256,
        virtual_batch_size=128
    )

    preds = model.predict(X_val_scaled.values)
    score = f1_score(y_val, preds)

    return score  # or ROC AUC


In [20]:
# 4. Run Tuning (the Optuna Study)

study = optuna.create_study(direction="maximize")  # Creates a new study
study.optimize(objective, n_trials=10, show_progress_bar=True)  # Run with progress bar


[I 2025-03-28 04:55:27,175] A new study created in memory with name: no-name-2afb2dc4-2a33-443a-97ee-0679dc2d69e6


  0%|          | 0/10 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.07094 | val_0_auc: 0.63635 | val_0_f1_score: 0.40426 |  0:00:23s
epoch 1  | loss: 0.02515 | val_0_auc: 0.66184 | val_0_f1_score: 0.35714 |  0:00:45s
epoch 2  | loss: 0.01868 | val_0_auc: 0.84247 | val_0_f1_score: 0.47619 |  0:01:08s
epoch 3  | loss: 0.01647 | val_0_auc: 0.83031 | val_0_f1_score: 0.672   |  0:01:31s
epoch 4  | loss: 0.01501 | val_0_auc: 0.84646 | val_0_f1_score: 0.70492 |  0:01:56s
epoch 5  | loss: 0.01433 | val_0_auc: 0.77067 | val_0_f1_score: 0.624   |  0:02:18s
epoch 6  | loss: 0.01348 | val_0_auc: 0.84022 | val_0_f1_score: 0.56075 |  0:02:43s
epoch 7  | loss: 0.0124  | val_0_auc: 0.91392 | val_0_f1_score: 0.73381 |  0:03:06s
epoch 8  | loss: 0.01147 | val_0_auc: 0.54064 | val_0_f1_score: 0.27184 |  0:03:27s
epoch 9  | loss: 0.01191 | val_0_auc: 0.77175 | val_0_f1_score: 0.35849 |  0:03:49s
epoch 10 | loss: 0.01019 | val_0_auc: 0.90572 | val_0_f1_score: 0.75385 |  0:04:11s
epoch 11 | loss: 0.00983 | val_0_auc: 0.9129  | val_0_f1_score: 0.69182 |  0

/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...

[I 2025-03-28 06:13:10,718] Trial 8 finished with value: 0.7230769230769231 and parameters: {'n_d': 64, 'n_a': 64, 'n_steps': 5, 'gamma': 1.5, 'lambda_sparse': 1.64947322619163e-05, 'lr': 0.00013093000549781525}. Best is trial 4 with value: 0.8421052631578947.


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.03955 | val_0_auc: 0.65386 | val_0_f1_score: 0.20455 |  0:00:13s
epoch 1  | loss: 0.01255 | val_0_auc: 0.59192 | val_0_f1_score: 0.26415 |  0:00:25s
epoch 2  | loss: 0.0095  | val_0_auc: 0.8388  | val_0_f1_score: 0.47273 |  0:00:38s
epoch 3  | loss: 0.00779 | val_0_auc: 0.89188 | val_0_f1_score: 0.47863 |  0:00:51s
epoch 4  | loss: 0.00651 | val_0_auc: 0.91396 | val_0_f1_score: 0.66102 |  0:01:04s
epoch 5  | loss: 0.00537 | val_0_auc: 0.87722 | val_0_f1_score: 0.59504 |  0:01:17s
epoch 6  | loss: 0.00594 | val_0_auc: 0.95052 | val_0_f1_score: 0.71429 |  0:01:30s
epoch 7  | loss: 0.00503 | val_0_auc: 0.9571  | val_0_f1_score: 0.69919 |  0:01:43s
epoch 8  | loss: 0.00437 | val_0_auc: 0.93685 | val_0_f1_score: 0.72581 |  0:01:57s
epoch 9  | loss: 0.00418 | val_0_auc: 0.94165 | val_0_f1_score: 0.7377  |  0:02:10s
epoch 10 | loss: 0.00411 | val_0_auc: 0.94447 | val_0_f1_score: 0.71429 |  0:02:23s
epoch 11 | loss: 0.0039  | val_0_auc: 0.96144 | val_0_f1_score: 0.752   |  0

/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


[I 2025-03-28 06:20:21,340] Trial 9 finished with value: 0.7874015748031497 and parameters: {'n_d': 32, 'n_a': 16, 'n_steps': 4, 'gamma': 2.0, 'lambda_sparse': 1.7746607930209065e-05, 'lr': 0.0008677580582246937}. Best is trial 4 with value: 0.8421052631578947.


In [26]:
import joblib
import pandas as pd
from utils.style_utils import styled_print 

# Save study object
joblib.dump(study, "tuning_results/tabnet_study.pkl")
styled_print("📂Saved Optuna study to: tuning_results/tabnet_study.pkl")

# Save best params
best_params = study.best_params
pd.DataFrame([best_params]).to_csv("tuning_results/tabnet_best_params.csv", index=False)
styled_print("📂Saved best params to: tuning_results/tabnet_best_params.csv")


## **2. Re-train TabNet on Train+Val using Best Params**

After finding the optimal configuration, we merged the training and validation sets to retrain the TabNet model from scratch with the best hyperparameters.

Key details:

- Model: TabNetClassifier with best parameters from Optuna

- Dataset: Full Train + Val (70% + 15% of original)

- Training setup: Same as before (50 epochs max, early stopping, batch size = 256)

**⚠️ Note: Test data was not used during tuning or retraining. It is held out for final evaluation only.**

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
import joblib
import numpy as np

styled_print(" Re-training Tabnet Model Using Best Parameters (NO test evaluation yet!)")
# Get best parameters from Optuna
best_params = study.best_params
styled_print(f" Best Params from Optuna: {best_params}")

# Merge train + val
X_trainval = np.vstack((X_train_scaled, X_val_scaled))
y_trainval = np.hstack((y_train, y_val))

# Rebuild model with best hyperparameters
tabnet_best = TabNetClassifier(
    n_d=best_params["n_d"],
    n_a=best_params["n_a"],
    n_steps=best_params["n_steps"],
    gamma=best_params["gamma"],
    lambda_sparse=best_params["lambda_sparse"],
    optimizer_params=dict(lr=best_params["lr"]),
    device_name="cpu",  
    verbose=1
)

# Retrain on full train+val
tabnet_best.fit(
    X_trainval, y_trainval,
    max_epochs=50,
    patience=10,
    batch_size=256,
    virtual_batch_size=128
)


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.14228 |  0:00:11s
epoch 1  | loss: 0.01925 |  0:00:23s
epoch 2  | loss: 0.01201 |  0:00:36s
epoch 3  | loss: 0.00975 |  0:00:47s
epoch 4  | loss: 0.00906 |  0:00:58s
epoch 5  | loss: 0.00835 |  0:01:09s
epoch 6  | loss: 0.00773 |  0:01:20s
epoch 7  | loss: 0.00762 |  0:01:32s
epoch 8  | loss: 0.00694 |  0:01:41s
epoch 9  | loss: 0.00671 |  0:01:51s
epoch 10 | loss: 0.00644 |  0:02:01s
epoch 11 | loss: 0.0063  |  0:02:12s
epoch 12 | loss: 0.00605 |  0:02:24s
epoch 13 | loss: 0.00575 |  0:02:35s
epoch 14 | loss: 0.00568 |  0:02:46s
epoch 15 | loss: 0.00551 |  0:02:56s
epoch 16 | loss: 0.00529 |  0:03:07s
epoch 17 | loss: 0.00553 |  0:03:18s
epoch 18 | loss: 0.00534 |  0:03:30s
epoch 19 | loss: 0.00529 |  0:03:40s
epoch 20 | loss: 0.00516 |  0:03:50s
epoch 21 | loss: 0.00497 |  0:04:00s
epoch 22 | loss: 0.00493 |  0:04:09s
epoch 23 | loss: 0.00478 |  0:04:19s
epoch 24 | loss: 0.0046  |  0:04:28s
epoch 25 | loss: 0.00475 |  0:04:38s
epoch 26 | loss: 0.00483 |  0:04:48s
e

In [33]:
# Save the final retrained TabNet model after tuning
tabnet_best.save_model("tuning_results/tabnet_best_model_after_tuning")
styled_print("📂 Saved final TabNet model after tuning to: tuning_results/tabnet_best_model_after_tuning.zip")


Successfully saved model at tuning_results/tabnet_best_model_after_tuning.zip


In [34]:
#  Save the training history dictionary if needed later
joblib.dump(tabnet_best.history, "tuning_results/tabnet_best_history_after_tuning.pkl")
styled_print("📂 Training history saved to tuning_results/tabnet_best_history_after_tuning.pkl")


## **3.Unit Testing**

In [39]:
import unittest
import numpy as np
import joblib
import torch
from sklearn.metrics import f1_score
from pytorch_tabnet.tab_model import TabNetClassifier

# Load dataset once for all tests
data_path = "../artifacts/tabnet/data_scaled_nosmote_for_tabnet.pkl"
X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test = joblib.load(data_path)

# Dummy trial class for objective test
class DummyTrial:
    def suggest_categorical(self, name, choices):
        return choices[0]
    def suggest_int(self, name, low, high):
        return low
    def suggest_float(self, name, low, high, step=None, log=False):
        return low

class TestTabNetTuning(unittest.TestCase):

    def test_objective_function_returns_float(self):
        """Test the Optuna objective function returns a float F1-score"""
        from tuning.tabnet_tuning import objective  # Adjust if needed
        trial = DummyTrial()
        result = objective(trial)
        self.assertIsInstance(result, float)
        self.assertGreaterEqual(result, 0)
        self.assertLessEqual(result, 1)

    def test_best_params_keys(self):
        """Test that the best params dictionary contains required keys"""
        from tuning_results.tabnet_study import study  # Assuming loaded
        best_params = study.best_params
        required_keys = {"n_d", "n_a", "n_steps", "gamma", "lambda_sparse", "lr"}
        self.assertTrue(required_keys.issubset(set(best_params.keys())))

    def test_model_trains_on_best_params(self):
        """Test that TabNetClassifier trains on full train+val without errors"""
        from tuning_results.tabnet_study import study
        best_params = study.best_params

        model = TabNetClassifier(
            n_d=best_params["n_d"],
            n_a=best_params["n_a"],
            n_steps=best_params["n_steps"],
            gamma=best_params["gamma"],
            lambda_sparse=best_params["lambda_sparse"],
            optimizer_params=dict(lr=best_params["lr"]),
            device_name="cpu"
        )

        # Merge train + val
        X_trainval = np.vstack((X_train_scaled, X_val_scaled))
        y_trainval = np.hstack((y_train, y_val))

        try:
            model.fit(
                X_trainval, y_trainval,
                max_epochs=1,  # Keep fast for unit test
                patience=2,
                batch_size=256,
                virtual_batch_size=128
            )
        except Exception as e:
            self.fail(f"Training crashed with error: {e}")
